In [17]:
import sys
sys.path.append('../')

In [18]:
import generation_utilities
import numpy as np
import librosa
from glob import glob
import random
import IPython.display as ipd
import soundfile as sf
import importlib
import ipywidgets as widgets
import numpy as np
importlib.reload(generation_utilities)


vqvae\diffusion_pytorch_model.safetensors not found
Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]


Fetching 7 files:  14%|█▍        | 1/7 [00:00<00:02,  2.42it/s]













































Fetching 7 files: 100%|██████████| 7/7 [00:31<00:00,  4.47s/it]
unet\diffusion_pytorch_model.safetensors not found


<module 'generation_utilities' from '..\\generation_utilities.py'>

In [19]:
music = glob("../DataSet/ts_db_separated/accompaniment/*.mp3")

## Create music dictionary with keys as song names and values as path
music_dict = {}
for song in music:
    song_name = song.split("\\")[-1].split(".")[0]
    music_dict[song_name] = song

music_dict

{'22': '../DataSet/ts_db_separated/accompaniment\\22.mp3',
 'anti hero': '../DataSet/ts_db_separated/accompaniment\\anti hero.mp3',
 'back to december': '../DataSet/ts_db_separated/accompaniment\\back to december.mp3',
 'blank space': '../DataSet/ts_db_separated/accompaniment\\blank space.mp3',
 'cardigan': '../DataSet/ts_db_separated/accompaniment\\cardigan.mp3',
 'delicate': '../DataSet/ts_db_separated/accompaniment\\delicate.mp3',
 'lover story': '../DataSet/ts_db_separated/accompaniment\\lover story.mp3',
 'lover': '../DataSet/ts_db_separated/accompaniment\\lover.mp3',
 'willow': '../DataSet/ts_db_separated/accompaniment\\willow.mp3',
 'you belong with me': '../DataSet/ts_db_separated/accompaniment\\you belong with me.mp3'}

In [20]:
def load_songs_as_numpy_array(song_dict):
    song_names = list(song_dict.keys())
    song_paths = list(song_dict.values())

    song_selector = widgets.SelectMultiple(
        options=song_names,
        description='Select songs:',
        rows=len(song_names),
        layout=widgets.Layout(width='400px')
    )

    ## Add a slider between 0 and 1 to select the similarity
    similarity_slider = widgets.FloatSlider(
        value=0.75,
        min=0,
        max=1.0,
        step=0.05,
        description='Similarity:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
    )


    select_button = widgets.Button(description='Select')
    output = widgets.Output()

    def on_select_button_clicked(_):
        selected_songs = [librosa.load(song_paths[i], sr=22050)[0] for i in song_selector.index]      
        name = '+'.join([song_names[i] for i in song_selector.index])

        with output:
            output.clear_output()
            print(f'Selected songs: {name}')
            ## Get similarity from the slider
            similarity = similarity_slider.value
            spec, song = generation_utilities.generate_songs(selected_songs, similarity=similarity, quality=500, merging_quality=100, device='cuda')

            ## Play all the selected songs
            print("Selected songs: ")
            for s in selected_songs:
                ipd.display(ipd.Audio(s, rate=22050))

            # Play song
            print("\n\nGenerated song: ")
            ipd.display(ipd.Audio(song, rate=22050))

            ## Get a rating from the user with ipywidgets
            rating = widgets.Text(
                value='',
                placeholder='Type something',
                description='Rating:',
                disabled=False
            )
            display(rating)
            ## Make a submit button
            submit_button = widgets.Button(description='Submit')
            display(submit_button)

            def on_submit_button_clicked(_):
                ## Convert rating to float
                rating_value = float(rating.value)

                ## Save the song
                total_files = glob('../DataSet/Generated_music_withratings/Vishwa_p2/Song/*')
                sf.write(f'../DataSet/Generated_music_withratings/Vishwa_p2/Song/{len(total_files)}_{name}_{similarity}_{rating_value}.wav', song, 22050, 'PCM_24')

                ## Save the spectrogram
                total_files = glob('../DataSet/Generated_music_withratings/Vishwa_p2/Spec/*')
                np.save(f'../DataSet/Generated_music_withratings/Vishwa_p2/Spec/{len(total_files)}_{name}_{similarity}_{rating_value}.npy', spec)

                with output:
                    output.clear_output()
                    print("Saved")


            submit_button.on_click(on_submit_button_clicked)
        
    select_button.on_click(on_select_button_clicked)

    display(song_selector)
    display(select_button)
    display(similarity_slider)
    display(output)


    # return selected_songs, name


In [21]:
load_songs_as_numpy_array(music_dict)

SelectMultiple(description='Select songs:', layout=Layout(width='400px'), options=('22', 'anti hero', 'back to…

Button(description='Select', style=ButtonStyle())

FloatSlider(value=0.75, continuous_update=False, description='Similarity:', max=1.0, step=0.05)

Output()